# 🐾 Multi-Class Pet Breed Classification (37 Classes)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/oxford-pets-classification/blob/main/notebooks/01_multiclass_classification.ipynb)

This notebook demonstrates multi-class classification for 37 different cat and dog breeds.

**What you'll see:**
- 🏗️ Custom CNN for 37-class classification  
- 📊 Confusion matrix analysis  
- 🎯 Per-class accuracy breakdown  
- 🖼️ Prediction visualization  

**Estimated time:** 60-90 minutes for full training (100 epochs)

## ⚙️ Setup

In [ ]:
# Clone repository
!git clone https://github.com/YOUR_USERNAME/oxford-pets-classification.git
%cd oxford-pets-classification

# Install dependencies
!pip install -q -r requirements.txt

In [ ]:
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

sys.path.insert(0, '.')

from configs.config import MultiClassConfig
from models.architectures import get_model, count_parameters
from utils.data_utils import prepare_multiclass_dataloaders, get_predictions
from utils.trainer import MultiClassTrainer
from utils.visualization import plot_training_curves, plot_confusion_matrix

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🖥️ Device: {device}")

## 🎛️ Configuration

In [ ]:
config = MultiClassConfig
config.EPOCHS = 10  # Use 100 for full training
config.BATCH_SIZE = 128
config.DEVICE = device
config.create_directories()

print(f"📋 Config: {config.NUM_CLASSES} classes, {config.EPOCHS} epochs")

## 📊 Load Data

In [ ]:
train_loader, val_loader, test_loader, class_names = prepare_multiclass_dataloaders(config)

print(f"Train: {len(train_loader.dataset):,}")
print(f"Val: {len(val_loader.dataset):,}")
print(f"Test: {len(test_loader.dataset):,}")
print(f"\nClasses: {len(class_names)}")
print("Sample breeds:", class_names[:10])

## 🧠 Create Model

In [ ]:
model = get_model('multiclass', num_classes=config.NUM_CLASSES)
model = model.to(config.DEVICE)

total_params, trainable_params = count_parameters(model)
print(f"Total parameters: {total_params:,}")

## 🚀 Training

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=config.LEARNING_RATE, weight_decay=config.WEIGHT_DECAY)

trainer = MultiClassTrainer(model, config.DEVICE, criterion, optimizer)

history = trainer.fit(train_loader, val_loader, epochs=config.EPOCHS, verbose=True)

## 📈 Results

In [ ]:
plot_training_curves(history, title="Multi-Class Classification")

test_loss, test_acc = trainer.evaluate(test_loader)
print(f"\nTest Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")

## 🎯 Confusion Matrix

In [ ]:
y_pred, y_true = get_predictions(model, test_loader, config.DEVICE)

plot_confusion_matrix(
    y_true, y_pred,
    class_names=class_names,
    normalize=True,
    figsize=(16, 14)
)

## 📊 Per-Class Accuracy

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)

# Extract per-class accuracy
class_accuracies = [(class_names[i], report[class_names[i]]['f1-score']) 
                    for i in range(len(class_names))]
class_accuracies.sort(key=lambda x: x[1], reverse=True)

# Plot top 10 and bottom 10
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Top 10
top_10 = class_accuracies[:10]
breeds, scores = zip(*top_10)
ax1.barh(breeds, scores, color='green', alpha=0.7)
ax1.set_xlabel('F1-Score')
ax1.set_title('Top 10 Best Performing Breeds', fontweight='bold')
ax1.invert_yaxis()

# Bottom 10
bottom_10 = class_accuracies[-10:]
breeds, scores = zip(*bottom_10)
ax2.barh(breeds, scores, color='red', alpha=0.7)
ax2.set_xlabel('F1-Score')
ax2.set_title('Top 10 Worst Performing Breeds', fontweight='bold')
ax2.invert_yaxis()

plt.tight_layout()
plt.show()

print(f"\nAverage F1-Score: {np.mean([x[1] for x in class_accuracies]):.4f}")

## 🎯 Conclusion

You've successfully trained a 37-class classifier!

**Next:** Try `02_transfer_learning.ipynb` for better accuracy with ResNet50.